In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## import datetime

# --- Data Store ---

inventory = {
    'coffee_beans_kg': {'quantity': 10.0, 'unit': 'kg'},
    'milk_liters': {'quantity': 20.0, 'unit': 'liters'},
    'sugar_kg': {'quantity': 5.0, 'unit': 'kg'},
    'syrup_bottles': {'quantity': 15, 'unit': 'bottles'},
    'cups': {'quantity': 100, 'unit': 'units'}
}

menu = {
    'espresso': {
        'price': 2.50,
        'recipe': {'coffee_beans_kg': 0.02, 'cups': 1}
    },
    'latte': {
        'price': 3.50,
        'recipe': {'coffee_beans_kg': 0.02, 'milk_liters': 0.15, 'cups': 1}
    },
    'cappuccino': {
        'price': 3.50,
        'recipe': {'coffee_beans_kg': 0.02, 'milk_liters': 0.1, 'cups': 1}
    },
    'americano': {
        'price': 3.00,
        'recipe': {'coffee_beans_kg': 0.02, 'cups': 1} # Water is assumed infinite
    },
    'vanilla_latte': {
        'price': 4.00,
        'recipe': {'coffee_beans_kg': 0.02, 'milk_liters': 0.15, 'syrup_bottles': 0.05, 'cups': 1}
    }
}

sales_log = []

# --- Core Functions ---

def check_inventory(order_item, quantity):
    """
    Checks if there are enough ingredients in inventory to fulfill an order.

    Args:
        order_item (str): The name of the menu item being ordered.
        quantity (int): The number of items being ordered.

    Returns:
        bool: True if ingredients are sufficient, False otherwise.
        list: A list of ingredients that are out of stock for this order.
    """
    if order_item not in menu:
        return False, ["Item not on menu"]

    recipe = menu[order_item]['recipe']
    missing_ingredients = []

    for ingredient, required_amount in recipe.items():
        total_required = required_amount * quantity
        if inventory[ingredient]['quantity'] < total_required:
            missing_ingredients.append(ingredient)

    if not missing_ingredients:
        return True, []
    else:
        return False, missing_ingredients

def process_sale(order_item, quantity):
    """
    Processes a sale by updating inventory and logging the transaction.

    Args:
        order_item (str): The name of the menu item sold.
        quantity (int): The number of items sold.

    Returns:
        float: The total cost of the sale, or 0.0 if the sale failed.
    """
    is_possible, missing = check_inventory(order_item, quantity)

    if not is_possible:
        print(f"\n[ERROR] Cannot complete order. Insufficient stock for: {', '.join(missing)}")
        return 0.0

    # Deduct ingredients from inventory
    recipe = menu[order_item]['recipe']
    for ingredient, required_amount in recipe.items():
        inventory[ingredient]['quantity'] -= required_amount * quantity

    # Log the sale
    item_price = menu[order_item]['price']
    total_cost = item_price * quantity
    sale_record = {
        'timestamp': datetime.datetime.now(),
        'item': order_item.title(),
        'quantity': quantity,
        'total_price': total_cost
    }
    sales_log.append(sale_record)

    print(f"\nOrder successful! Total cost: ${total_cost:.2f}")
    return total_cost

def restock_inventory(item_name, amount):
    """
    Adds a specified amount to an inventory item's quantity.

    Args:
        item_name (str): The key of the inventory item to restock.
        amount (float): The quantity to add.
    """
    if item_name in inventory:
        inventory[item_name]['quantity'] += amount
        print(f"\nRestocked {amount} {inventory[item_name]['unit']} of {item_name}. New total: {inventory[item_name]['quantity']:.2f}")
    else:
        print(f"\n[ERROR] Inventory item '{item_name}' not found.")

# --- User Interface Functions ---

def display_coffee_menu():
    """Prints the coffee menu in a formatted way."""
    print("\n--- Coffee Menu ---")
    for item, details in menu.items():
        print(f"{item.title():<15} | ${details['price']:.2f}")
    print("--------------------")

def handle_order():
    """Handles the user interaction for placing a new order."""
    display_coffee_menu()
    try:
        order_item = input("What would you like to order? ").lower().strip()
        if order_item not in menu:
            print("\n[ERROR] Sorry, that item is not on our menu.")
            return

        quantity_str = input(f"How many '{order_item.title()}' would you like? ")
        quantity = int(quantity_str)
        if quantity <= 0:
            print("\n[ERROR] Quantity must be a positive number.")
            return

        process_sale(order_item, quantity)

    except ValueError:
        print("\n[ERROR] Invalid input. Please enter a valid number for quantity.")
    except Exception as e:
        print(f"\n[ERROR] An unexpected error occurred: {e}")

def display_inventory():
    """Prints the current inventory levels."""
    print("\n--- Current Inventory ---")
    for item, details in inventory.items():
        print(f"{item:<18} | {details['quantity']:.2f} {details['unit']}")
    print("-------------------------")

def handle_restock():
    """Handles the user interaction for restocking inventory."""
    display_inventory()
    try:
        item_to_restock = input("Enter the inventory item name to restock (e.g., coffee_beans_kg): ").lower().strip()
        if item_to_restock not in inventory:
            print("\n[ERROR] That item does not exist in the inventory.")
            return

        amount_str = input(f"Enter the amount of '{item_to_restock}' to add: ")
        amount = float(amount_str)
        if amount <= 0:
            print("\n[ERROR] Restock amount must be positive.")
            return

        restock_inventory(item_to_restock, amount)

    except ValueError:
        print("\n[ERROR] Invalid input. Please enter a valid number for the amount.")
    except Exception as e:
        print(f"\n[ERROR] An unexpected error occurred: {e}")

def display_sales_report():
    """Calculates and prints a summary of all sales."""
    print("\n--- Daily Sales Report ---")
    if not sales_log:
        print("No sales recorded yet.")
        print("------------------------")
        return

    total_revenue = 0.0
    items_sold = {}

    for sale in sales_log:
        total_revenue += sale['total_price']
        item = sale['item']
        quantity = sale['quantity']
        items_sold[item] = items_sold.get(item, 0) + quantity

    print(f"Total Revenue: ${total_revenue:.2f}\n")
    print("Items Sold:")
    if not items_sold:
        print("  None")
    else:
        for item, count in items_sold.items():
            print(f"  - {item}: {count}")

    print("------------------------")

def display_main_menu():
    """Prints the main menu options for the CLI."""
    print("\n===== Coffee Shop Management System =====")
    print("1. Place a New Order")
    print("2. View Current Inventory")
    print("3. Restock Inventory")
    print("4. View Sales Report")
    print("5. Exit")
    print("=======================================")

# --- Main Application Loop ---

def main():
    """The main function to run the coffee shop CLI."""
    while True:
        display_main_menu()
        choice = input("Please select an option (1-5): ")

        if choice == '1':
            handle_order()
        elif choice == '2':
            display_inventory()
        elif choice == '3':
            handle_restock()
        elif choice == '4':
            display_sales_report()
        elif choice == '5':
            print("\nThank you for using the system. Goodbye!")
            break
        else:
            print("\n[ERROR] Invalid choice. Please enter a number between 1 and 5.")

if __name__ == "__main__":
    main()